В данной лабораторной работе будут показаны основы анализа текстовой информации. В ходе её выполнения мы познакомимся с этапами предварительной подготовки данных, а также применим машинное обучение для задачи классификации.

Будем использовать датасет с текстами песен разных жанров: https://www.kaggle.com/mehedihasan9021/movie-script-dataset


> Задание выполняется в google colab. Чтобы редактировать ноутбук, не забудьте сохранить его копию на диске. Скачайте файл по ссылке, загрузите в сессионное хранилище и запускайте ячейки с кодом, следуя инструкциям.

Читаем набор данных

# 0.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")

Mounted at /content/drive/


In [ ]:
data.head()

,genre,lyrics,SongInfo
0,Christian,"Who am I, that the Lord of all the earth Woul...",CASTING CROWNS - WHO AM I LYRICS
1,Christian,Glory Revealed By His Wounds He was pierced ...,GLORY REVEALED - BY HIS WOUNDS LYRICS
2,Christian,Lord of heaven and earth Lord of all creation...,CAEDMON'S CALL - GOD OF WONDERS LYRICS
3,Christian,I can only imagine what it will be like When ...,MERCYME - I CAN ONLY IMAGINE LYRICS
4,Christian,I am not skilled to understand What God has w...,AARON SHUST - MY SAVIOR MY GOD LYRICS


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558 entries, 0 to 557
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     558 non-null    object
 1   lyrics    558 non-null    object
 2   SongInfo  558 non-null    object
dtypes: object(3)
memory usage: 13.2+ KB


In [ ]:
columns = data[['genre', 'lyrics']]

In [ ]:
columns

,genre,lyrics
0,Christian,"Who am I, that the Lord of all the earth Woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...
3,Christian,I can only imagine what it will be like When ...
4,Christian,I am not skilled to understand What God has w...
...,...,...
553,R&B,"Ha I dont care ha, about your past I just wan..."
554,R&B,Hoverin by my suitcase Tryin to find a warm ...
555,R&B,I dont know why I love you like I do After a...
556,R&B,"C. C. Rider Elvis Presley Well now see., C. ..."


Посмотрим какие жанры присутствуют в датасете. Для этого выведем уникальные значения столбца:

In [ ]:
columns['genre'].unique()

array(['Christian', 'Country', 'Hip-Hop', 'Pop', 'Rock', 'R&B'],
      dtype=object)

Посчитаем также сколько песен каждого жанра представлено:

In [ ]:
columns['genre'].value_counts()

Pop          100
Rock          95
Christian     94
R&B           91
Hip-Hop       91
Country       87
Name: genre, dtype: int64

Оставляем только 2 жанра, которые планируем отличать друг от друга. Для примера будут использованы Christian и Hip-Hop, позднее в формулировке задания нужно будет сгенерировать собственную пару жанров. 

In [ ]:
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Hip-Hop')]

###Предварительная обработка текстовых данных

Для того, чтобы применять текстовые данные для обучения той или иной модели машинного обучения, их нужно привести в подходящий для этого вид. В этом нам помогут следующие шаги предварительной подготовки:

####Приведение к нижнему регистру

Так как слова (например) "Beer" и "beer" будут считаться разными словами из-за разницы в регистре буквы b, а первая буква предложения, как правило, является заглавной, важно привести все слова предложения к нижнему регистру. Таким образом, "beer", встретившееся в начале предложения и в его середине, будут распознаны как одно слово.

In [ ]:
lowered = columns['lyrics'].str.lower()

In [ ]:
columns['lowered'] = lowered

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Токенизация

Имеющиеся тексты нужно разбить на отдельные слова. Такой процесс называется ***токенизация***.

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)

In [ ]:
columns['tokened'] = tokened

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Удаление стоп-слов

***Стоп-словами*** называются распространённые слова, не несущие особой смысловой нагрузки, а значит никак не помогающие в последующей задаче классификации. Такие слова мы считаем шумом, и, соответственно, удаляем.

Существует специальный корпус стоп-слов на разных языках. Так как сейчас мы имеем дело с текстами на английском языке, выведем стоп-слова для английского:

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
noise = stopwords.words('english')

In [ ]:
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])

In [ ]:
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))

In [ ]:
columns['without_stop'] = without_stop

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


####Лемматизация

У одного и того же слова бывают разные формы. Например, dances - форма слова dance. Лемматизация - это приведение к ***лемме*** - исходной форме слова. Её также необходимо применить к нашим данным, чтобы разные формы слова не считались за отдельные слова.

Существует целый ряд лемматизаторов, которые показывают себя с разной эффективностью на разных языках. Для работы с английским языком эффективен **WordNetLemmatizer**

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
print(lemmatizer.lemmatize("dances"))

dance


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])

In [ ]:
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))

In [ ]:
columns['lemmatized'] = lemma

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Таким образом, на каждом этапе мы добавляли столбец с той или иной модификацией, чтобы получить пригодные для обучения данные. Посмотрим на датафрейм, который получился:

In [ ]:
columns

,genre,lyrics,lowered,tokened,without_stop,lemmatized
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul...","[who, am, i, ,, that, the, lord, of, all, the,...",",, lord, earth, would, care, know, name, would...",",, lord, earth, would, care, know, name, would..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...,"[glory, revealed, by, his, wounds, he, was, pi...","glory, revealed, wounds, pierced, transgressio...","glory, revealed, wounds, pierced, transgressio..."
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...,"[lord, of, heaven, and, earth, lord, of, all, ...","lord, heaven, earth, lord, creation, lord, hea...","lord, heaven, earth, lord, creation, lord, hea..."
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...,"[i, can, only, imagine, what, it, will, be, li...","imagine, like, walk, side, imagine, eyes, see,...","imagine, like, walk, side, imagine, eyes, see,..."
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...,"[i, am, not, skilled, to, understand, what, go...","skilled, understand, god, willed, ,, god, plan...","skilled, understand, god, willed, ,, god, plan..."
...,...,...,...,...,...,...
267,Hip-Hop,"Dirty dog Im, Im a dirty dog Im a dirty dog I...","dirty dog im, im a dirty dog im a dirty dog i...","[dirty, dog, im, ,, im, a, dirty, dog, im, a, ...","dirty, dog, im, ,, im, dirty, dog, im, dirty, ...","dirty, dog, im, ,, im, dirty, dog, im, dirty, ..."
268,Hip-Hop,"Regulators, we regulate any stealing of his p...","regulators, we regulate any stealing of his p...","[regulators, ,, we, regulate, any, stealing, o...","regulators, ,, regulate, stealing, property, d...","regulators, ,, regulate, stealing, property, d..."
269,Hip-Hop,Have you ever met a girl that you tried to da...,have you ever met a girl that you tried to da...,"[have, you, ever, met, a, girl, that, you, tri...","ever, met, girl, tried, date, year, make, love...","ever, met, girl, tried, date, year, make, love..."
270,Hip-Hop,"Yo, yo, yo They wanna know Whos that girl? (...","yo, yo, yo they wanna know whos that girl? (...","[yo, ,, yo, ,, yo, they, wan, na, know, whos, ...","yo, ,, yo, ,, yo, wan, na, know, whos, girl, ?...","yo, ,, yo, ,, yo, wan, na, know, whos, girl, ?..."


####Разделим данные на обучающую и тестовую выборки

x_train - тексты, на которых мы обучаем модель. В данном случае мы используем столбец lemmatized, так как он содержит данные, прошедшие все этапы подготовки 

> 



y_train - жанры, соответствующие текстам, на которых модель обучается - столбец genre


> 


x_test - точно такие же тексты из набора данных, на которых мы будем проверять, насколько модель научилась предсказывать жанр


> 



y_test - жанры, соответствующие x_test. т.е. мы смотрим, насколько предсказания соответствуют содержимому этой переменной, чтобы оценить качество обучения

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)

In [ ]:
columns.genre.value_counts()


Christian    94
Hip-Hop      91
Name: genre, dtype: int64

####Векторизация

Следующий этап - ***векторизация***, то есть представление текста в численном виде, чтобы закодированные данные в дальнейшем использовать на модели.

Иногда помимо отдельных слов для улучшения качества обучения будет полезно использовать также комбинации из 2-3 слов. Здесь мы используем векторизатор CountVectorizer, который имеет встроенный метод - мешок n-грамм.

> n = 1 - униграмма (для обучения используются слова по отдельности)

> n = 2 - биграмма (для обучения используются пары слов)

> n = 3 - триграмма (для обучения используются тройки слов)







In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Задаём для векторизатора ngram_range=(1, 3), то есть используем все варианты n от 1 до 3 и прибегаем как к униграммам, так и к биграммам и триграммам:

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

### Классификация

Для задачи классификации используем Наивный Байесовский Классификатор - простой вероятностный классификатор, основанный на применении теоремы Байеса со строгими предположениями о независимости. 

In [ ]:
#импортируем байесовский классификатор
from sklearn.naive_bayes import MultinomialNB 

In [ ]:
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
# тестовую выборку просто векторизировали
vectorized_x_test = vectorizer.transform(x_test)

Посмотрим предсказания для тестовой выборки

In [ ]:
clf.predict(vectorized_x_test)

array(['Hip-Hop', 'Christian', 'Christian', 'Christian', 'Christian',
       'Hip-Hop', 'Christian', 'Hip-Hop', 'Christian', 'Christian',
       'Hip-Hop', 'Christian', 'Christian', 'Hip-Hop', 'Christian',
       'Hip-Hop', 'Christian', 'Christian', 'Hip-Hop', 'Hip-Hop',
       'Christian', 'Hip-Hop', 'Hip-Hop', 'Christian', 'Christian',
       'Christian', 'Christian', 'Hip-Hop', 'Hip-Hop', 'Christian',
       'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop',
       'Christian', 'Hip-Hop', 'Christian', 'Christian', 'Hip-Hop',
       'Christian', 'Hip-Hop', 'Christian', 'Hip-Hop', 'Christian',
       'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Hip-Hop', 'Christian',
       'Christian', 'Christian', 'Christian', 'Hip-Hop'], dtype='<U9')

Получим оценки классификации:

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.96      0.90      0.93        30
     Hip-Hop       0.89      0.96      0.93        26

    accuracy                           0.93        56
   macro avg       0.93      0.93      0.93        56
weighted avg       0.93      0.93      0.93        56



###Задание

# 1.

1) Запустить ячейку ниже, чтобы получить 2 жанра. Для полученных жанров провести все этапы предварительной обработки текста (как в примере), обучить наивный байесовский классификатор, численно оценить его работу.

In [ ]:
 import random 
 lst = ['Christian', 'Country', 'Pop', 'Rock', 'R&B'] 
 print('ваши жанры', random.choice(lst), 'и', random.choice(lst)) 

ваши жанры Pop и Christian


In [ ]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")
columns = data[['genre', 'lyrics']]
columns = columns[(columns.genre == 'Christian') | (columns.genre == 'Pop')]

columns['lowered'] = columns['lyrics'].str.lower()
columns

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


,genre,lyrics,lowered
0,Christian,"Who am I, that the Lord of all the earth Woul...","who am i, that the lord of all the earth woul..."
1,Christian,Glory Revealed By His Wounds He was pierced ...,glory revealed by his wounds he was pierced ...
2,Christian,Lord of heaven and earth Lord of all creation...,lord of heaven and earth lord of all creation...
3,Christian,I can only imagine what it will be like When ...,i can only imagine what it will be like when ...
4,Christian,I am not skilled to understand What God has w...,i am not skilled to understand what god has w...
...,...,...,...
367,Pop,Have you ever met a girl that you tried to da...,have you ever met a girl that you tried to da...
368,Pop,"Im outta luck, outta love Gotta photograph, p...","im outta luck, outta love gotta photograph, p..."
369,Pop,She was a fast machine She kept her motor cl...,she was a fast machine she kept her motor cl...
370,Pop,"Good times, these are the good times Leave yo...","good times, these are the good times leave yo..."


*Токенизация*

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


NameError: ignored

Удаление стоп-слов

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma    

Разделение данных

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.genre, train_size = 0.7)
columns.genre.value_counts()

Pop          100
Christian     94
Name: genre, dtype: int64

Векторизация

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

   Christian       0.93      0.90      0.92        30
         Pop       0.90      0.93      0.92        29

    accuracy                           0.92        59
   macro avg       0.92      0.92      0.92        59
weighted avg       0.92      0.92      0.92        59



# 2.

2) Найти (нагуглить) по песне каждого из жанров, которые Вам достались, после необходимой обработки их текстов определить жанр обеих песен с помощью обученной в ходе выполнения предыдущего пункта модели.

In [ ]:
music = ['I can only imagine What it will be like When I walk by Your side I can only imagine What my eyes would see When Your face is before me I can only imagine Yeah Surrounded by Your glory What will my heart feel? Will I dance for You Jesus Or in awe of You be still? Will I stand in Your presence Or to my knees, will I fall? Will I sing hallelujah? Will I be able to speak at all? I can only imagine I can only imagine I can only imagine When that day comes And I find myself Standing in the Son I can only imagine When all I will do Is forever, forever worship You I can only imagine, yeah I can only imagine Surrounded by Your glory What will my heart feel? Will I dance for you Jesus Or in awe of You be still? Will I stand in your presence Or to my knees will I fall? Will I sing hallelujah? Will I be able to speak at all? I can only imagine, yeah I can only imagine Surrounded by Your glory What will my heart feel? Will I dance for you Jesus Or in awe of You be still? Will I stand in your presence Or to my knees, will I fall? Will I sing hallelujah? Will I be able to speak at all? I can only imagine, yeah I can only imagine I can only imagine, yeah-yeah I can only imagine I can only imagine I can only imagine I can only imagine When all I will do Is forever, forever worship You I can only imagine', 
            'Oh I cannot explain Every time it is the same Oh I feel that it is real Take my heart I have been lonely too long Oh I can not be so strong Take a chance for romance Take my heart I need you so There is no time I will ever go Cheri Cheri Lady Going through emotion Love is where you find it Listen to your heart Cheri Cheri Lady Living in devotion Always like the first time Let me take a part Cheri Cheri Lady Like there is no tomorrow Take my heart don not lose it Listen to your heart Cheri Cheri Lady To know you is to love you If you call me baby I will be always yours I get up and get down Oh my world turns around Who is right, who is wrong I don not know I have got pain in my heart Gotta love in my soul Easy come but I think easy go I need you so All those times I am not so strong Cheri Cheri Lady Going through emotion Love is where you find it Listen to your heart Cheri Cheri Lady Living in devotion Always like the first time Let me take a part Cheri Cheri Lady Like there is no tomorrow Take my heart don not lose it Listen to your heart Cheri Cheri Lady To know you is to love you If you call me baby I will be always yours'] 
#нижний регистр
music_lower = list(map(str.lower, music))
#токенизация
music_tok = [nltk.word_tokenize(i) for i in music_lower]
music_tokened = list(flatten(music_tok))
#удаление стоп-слов
music_withoutstop = list(set(music_tokened) - set(noise))
#лемматизация
music_pro = [lemmatizer.lemmatize(i) for i in music_withoutstop]

#векторизация
vectorized_mus = vectorizer.transform(music) 
clf.predict(vectorized_mus)


array(['Christian', 'Pop'], dtype='<U9')

# 3.

3) С помощью набора данных по ссылке аналогичным образом научить модель отличать тексты песен Дэвида Боуи от текстов песен Пола МакКартни 
https://www.kaggle.com/italomarcelo/dataset-lyrics-music-mini

In [ ]:
import pandas as pd
from pandas.core.common import flatten
from google.colab import drive
drive.mount('/content/drive/')
dat = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset-lyrics-musics-mini.csv")
columns = dat[['cantorNome', 'letra']]
columns = columns[(columns.cantorNome == 'david-bowie') | (columns.cantorNome == 'paul-mccartney')]

columns['lowered'] = columns['letra'].str.lower()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


*Токенизация*

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokened = columns.apply(lambda row: nltk.word_tokenize(row['lowered']), axis=1)
columns['tokened'] = tokened

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Удаление стоп-слов

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
noise = stopwords.words('english')
withoutstop = columns['tokened'].apply(lambda x: [item for item in x if item not in noise])
without_stop = []
for a in withoutstop:    
    without_stop.append(", ".join(a))
columns['without_stop'] = without_stop

Лемматизация

In [ ]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
lemmatized = columns['without_stop'].apply(lambda x: [lemmatizer.lemmatize(x)])
lemma = []
for a in lemmatized:    
    lemma.append(", ".join(a))
columns['lemmatized'] = lemma    

Разделение данных

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(columns.lemmatized, columns.cantorNome, train_size = 0.7)
columns.cantorNome.value_counts()

david-bowie       483
paul-mccartney    464
Name: cantorNome, dtype: int64

Векторизация

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3))
vectorized_x_train = vectorizer.fit_transform(x_train)

Классификация

In [ ]:
from sklearn.naive_bayes import MultinomialNB 
clf = MultinomialNB()
clf.fit(vectorized_x_train, y_train)
vectorized_x_test = vectorizer.transform(x_test)

Оценка классификации

In [ ]:
from sklearn.metrics import * 
pred = clf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

                precision    recall  f1-score   support

   david-bowie       0.73      0.74      0.73       141
paul-mccartney       0.74      0.74      0.74       144

      accuracy                           0.74       285
     macro avg       0.74      0.74      0.74       285
  weighted avg       0.74      0.74      0.74       285

